In [1]:
import camelot
import json
import re
import datetime
import pandas as pd

from bs4 import BeautifulSoup
import requests
import datetime

In [2]:
ru_url = 'https://ru.unb.br'
menu_url = ru_url + '/index.php/cardapio-refeitorio'

html_menu_response = requests.get(menu_url)

In [3]:
menu_soup = BeautifulSoup(html_menu_response.text, 'html')

campuses_names = []
campuses = menu_soup('strong')
for campus in campuses:
   if isinstance(campus.contents[0], str):
    campuses_names.append(campus.string)
print(campuses_names)

available_menu_links = menu_soup.find('main').find_all('a')
tables = {}
idx = 0
d = datetime.datetime.now()
for item in available_menu_links:
    if item.contents and item.string and item.string != 'I' and '22' in item.string: # m.contents[0] != 'I', isso é para desconsiderar um link nada a ver no site
      menu_link = ru_url + item.attrs['href']
      print('Processando cardapio: ' + menu_link)
      tables[campuses_names[idx]] = camelot.read_pdf(menu_link, pages='all', flavor='lattice', line_scale=50, strip_text='\n', copy_text=['h'], )
      idx += 1
tables['Darcy Ribeiro'][1].df
tables['Ceilândia'][1].df

['Darcy Ribeiro', 'Ceilândia', 'Gama', 'Planaltina', 'Fazenda\xa0']
Processando cardapio: https://ru.unb.br/images/Artigos/0ISMCARDAPIOSDRU2024/7JULHO2024/Darcy_Ribeiro_-_Semana_05_-_22-07_a_28-07.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/0ISMCARDAPIOSDRU2024/7JULHO2024/Ceilndia_-_Semana_05_-_22-07_a_28-07.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/0ISMCARDAPIOSDRU2024/7JULHO2024/Gama_-_Semana_05_-_22-07_a_28-07.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/0ISMCARDAPIOSDRU2024/7JULHO2024/Planaltina_-_Semana_05_-_22-07_a_28-07.pdf
Processando cardapio: https://ru.unb.br/images/Artigos/0ISMCARDAPIOSDRU2024/7JULHO2024/Fazenda_-_Semana_05_-_22-07_a_28-07.pdf


,0,1,2,3,4,5,6,7
0,Almoço,COMPOSIÇÃO,2ª FEIRA 22/07/2024,3ª FEIRA 23/07/2024,4ª FEIRA 24/07/2024,5ª FEIRA 25/07/2024,6ª FEIRA 26/07/2024,SÁBADO 27/07/2024
1,,SALADA 1,Alface crespa com agrião,Mix de folhas,Acelga temperada,Alface crespa com rúcula,Alface roxa com mostarda,Alface lisa
2,,SALADA 2,Picles de abobrinha,"Tomate, cebola e pimentão em cubos",Cenoura ralada,"Salada de tomate, palmito e azeitona",Feijão fradinho com vinagrete,Chuchu cozido temperado
3,,MOLHO PARA SALADA,Alho,Ervas finas,Salsa / Limão,Chipotle,Ácido / Limão,Manjericão
4,,PRATO PRINCIPAL PADRÃO,Escondidinho de carne moída,Frango assado ao molho de salsa,Suíno ao molho de ervas OPÇÃO: Bife acebolado,Strogonoff de frango,Peixe ao molho de moqueca OPÇÃO: Carne ao molh...,Frango assado com mostarda e mel
5,,PRATO PRINCIPAL OVOLACTOVEGETARIANO,Refogado de vegetais com quinoa e ovos,Risoto de brócolis com queijo e soja,Fritata de vegetais,Berinjela recheada com ervilha e queijo gratinado,Bife de soja ao sugo e queijo,Lasanha de espinafre gratinada
6,,PRATO PRINCIPAL VEGETARIANO ESTRITO,Yakissoba vegetariano,"Moqueca de vegetais com palmito, castanha e soja",Guisado de grão de bico,Espaguete com abobrinha e ervilha ao sugo,Almôndega de soja ao sugo,Charuto de couve com soja
7,,GUARNIÇÃO,Chuchu e cenoura refogados,Macarrão com alho e manjerição,Repolho com passas e maçã,Batata canoa sautê,Cenoura com salsinha e azeite,Purê de abóbora
8,,ACOMPANHAMENTOS,Arroz branco e integral Feijão carioca,Arroz branco e integral Feijão preto,Arroz branco e integral Feijão carioca,Arroz branco e integral Feijão preto,Arroz branco e integral Feijão carioca,Arroz branco e Arroz com cenoura Feijão preto
9,,SOBREMESA,Tangerina,Maçã,Doce de goiaba OPÇÃO: Fruta,Mamão,Banana,Laranja


In [4]:
def table_to_json(table):
  for i in range(len(table)):
    table[i].df = table[i].df.drop(0, axis=1) #removes first column
    table[i].df.index = table[i].df[1]
    table[i].df = table[i].df.drop(1, axis=1) #removes first column

  dates = re.findall(r'\d\d.\d\d.\d\d\d\d', table[0].df.to_string())
  dates_backlash_removed = map(lambda date: date.replace('/', ''), dates)
  formated_dates = map(lambda date: datetime.datetime.strptime(date, "%d%m%Y").strftime("%Y-%m-%d"), dates_backlash_removed)
  dates = list(formated_dates)

  x = table[0].df.T.values[i][1:].tolist()
  try:
    x.remove('Café OU chá')
  except:
    try:
      x.remove('Café ou chá')
    except:
      pass

  desjejum = []
  almoco = []
  jantar = []
  for j in range(len(table)):
    for i in range(len(table[j].df.columns.values)):
      if j == 0:
        desjejum_list = table[0].df.T.values[i][1:].tolist()
        try:
          desjejum_list.remove('Café OU chá')
        except:
          try:
            desjejum_list.remove('Café ou chá')
          except:
            pass
        desjejum.append(json.dumps(dict(zip(map(lambda x: x.capitalize(),list(filter(None, table[0].df.T.columns[1:]))), desjejum_list)), ensure_ascii=False))
      elif j == 1:
        print(table[1].df.T)
        almoco.append(json.dumps(dict(zip(map(lambda x: x.capitalize(), table[1].df.T.columns[1:]), table[1].df.T.values[i][1:])), ensure_ascii=False))
      elif j == 2:
        jantar.append(json.dumps(dict(zip(map(lambda x: x.capitalize(), table[2].df.T.columns[1:]), table[2].df.T.values[i][1:])), ensure_ascii=False))
  
  print(len(dates), len(desjejum), len(almoco), len(jantar))

  print(desjejum)
  print(almoco)
  print(jantar)
  return  pd.DataFrame({'date': dates, 'desjejum': desjejum, 'almoco': almoco, "jantar": jantar})

In [5]:
for campus in tables:
  res = table_to_json(tables[campus])
  res.to_csv(campus + ".csv", index=False)

1                                                   \
2             COMPOSIÇÃO                  SALADA 1   
3    2ª FEIRA 22/07/2024  Alface crespa com agrião   
4    3ª FEIRA 23/07/2024             Mix de folhas   
5    4ª FEIRA 24/07/2024          Acelga temperada   
6    5ª FEIRA 25/07/2024  Alface crespa com rúcula   
7    6ª FEIRA 26/07/2024  Alface roxa com mostarda   
8      SÁBADO 27/07/2024               Alface lisa   
9     DOMINGO 28/07/2024             Mix de folhas   

1                                                            \
2                               SALADA 2  MOLHO PARA SALADA   
3                    Picles de abobrinha               Alho   
4     Tomate, cebola e pimentão em cubos        Ervas finas   
5                         Cenoura ralada      Salsa / Limão   
6  Salada de tomate,  palmito e azeitona           Chipotle   
7          Feijão fradinho com vinagrete      Ácido / Limão   
8                Chuchu cozido temperado         Manjericão   
9        

ValueError: All arrays must be of the same length